在PyTorch中进行validation/test时，会使用model.eval()切换到测试模式，在该模式下：

1.主要用于通知dropout层和BN层在training和validation/test模式间切换：

在train模式下，dropout网络层会按照设定的参数p，设置保留激活单元的概率（保留概率=p)。BN层会继续计算数据的mean和var等参数并更新。

在eval模式下，dropout层会让所有的激活单元都通过，而BN层会停止计算和更新mean和var，直接使用在训练阶段已经学出的mean和var值。


2.eval模式不会影响各层的gradient计算行为，即gradient计算和存储与training模式一样，只是不进行反向传播（back probagation)。

而with torch.no_grad()则主要是用于停止autograd模块的工作，以起到加速和节省显存的作用。它的作用是将该with语句包裹起来的部分停止梯度的更新，从而节省了GPU算力和显存，但是并不会影响dropout和BN层的行为。

如果不在意显存大小和计算时间的话，仅仅使用model.eval()已足够得到正确的validation/test的结果；而with torch.no_grad()则是更进一步加速和节省gpu空间（因为不用计算和存储梯度），从而可以更快计算，也可以跑更大的batch来测试。


model的属性：

named_children: 

list(vgg19.named_children())

for params in model.parameters():
print('params: ',params)

vgg19.state_dict()

In [ ]:
#读取已经封装好的数据集
transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize(0.5, 0.5)]
)
# 如果没有下载 MNIST 数据集，那么需要设置 download 参数为 True
# 如果已经下载 MNIST 数据集，那么只需设置 download 参数为 False
trainset = torchvision.datasets.MNIST(root='./data', 
                                    train=True, 
                                    transform=transform, 
                                    download=False)
trainloader = torch.utils.data.DataLoader(dataset=trainset, 
                                            batch_size=batch_size, 
                                            shuffle=True, 
                                            num_workers=2)
testset = torchvision.datasets.MNIST(root='./data', 
                                    train=False, 
                                    transform=transform, 
                                    download=False)
testloader = torch.utils.data.DataLoader(dataset=testset, 
                                            batch_size=batch_size, 
                                            shuffle=True, 
                                            num_workers=2)

In [ ]:
#读取自己的数据集
class DIV2KDataset(data.Dataset):
    def __init__(self, file_path=[], crop_size_img=None, crop_size_label=None):
        """para:
            file_path(list): 数据和标签路径,列表元素第一个为图片路径，第二个为标签路径
        """
        # 1 正确读入图片和标签路径
        if len(file_path) != 2:
            raise ValueError("同时需要图片和标签文件夹的路径，图片路径在前")
        self.img_path = file_path[0]
        self.label_path = file_path[1]
        # 2 从路径中取出图片和标签数据的文件名保持到两个列表当中（程序中的数据来源）
        self.imgs = self.read_file(self.img_path)
        self.labels = self.read_file(self.label_path)
        # 3 初始化数据处理函数设置
        self.crop_size_img = crop_size_img
        self.crop_size_label = crop_size_label

    def __getitem__(self, index):
        img = self.imgs[index]
        label = self.labels[index]
        # 从文件名中读取数据（图片和标签都是png格式的图像数据）
        img = Image.open(img)
        label = Image.open(label)

        img, label = self.center_crop(img, label, crop_size_img, crop_size_label)

        img, label = self.img_transform(img, label)
        # print('处理后的图片和标签大小：',img.shape, label.shape)
        sample = {'img': img, 'label': label}

        return sample

    def __len__(self):
        return len(self.imgs)

    def read_file(self, path):
        """从文件夹中读取数据"""
        files_list = os.listdir(path)
        file_path_list = [os.path.join(path, img) for img in files_list]
        file_path_list.sort()
        return file_path_list

    def center_crop(self, data, label, crop_size_img, crop_size_label):
        """裁剪输入的图片和标签大小"""
        data = ff.center_crop(data, crop_size_img)
        label = ff.center_crop(label, crop_size_label)
        return data, label

    def img_transform(self, img, label):
        """对图片和标签做一些数值处理"""
        transform = transforms.Compose(
            [
                transforms.ToTensor(),
                # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]
        )
        img = transform(img)
        label = transform(label)

        return img, label

In [ ]:
# 训练数据集的加载器，自动将数据分割成batch，顺序随机打乱
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                            drop_last = True ,      
                                           shuffle=True)
————————————————
版权声明：本文为CSDN博主「tsz danger」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/weixin_43914889/article/details/104677675